In [13]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from langdetect import detect
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

ValueError: BLIS support requires blis: pip install blis

# Step 1: Data Extraction
This step involves extracting raw data from various file formats, such as PDFs and CSV files, and saving the data in a structured format (JSON).

Libraries and Technologies Used:
pdfminer.six (for extracting text from PDF files):

**Purpose**: pdfminer.six is a Python library used to extract text from PDF files. It parses the PDF content and converts it into a string format.

**Usage**: Extracts text from the Affiliated-Colleges.pdf and UOK-Admission Notification 2022-23.pdf.

**Documentation**: pdfminer.six
pandas (for extracting and reading CSV files):

**Purpose**: pandas is a powerful library for data manipulation and analysis. It can handle structured data and is used here to read the uok_data.csv.txt file.

**Usage**: Reads CSV files and extracts the contents as a string.
Documentation: pandas
json (for saving extracted data into a structured format):

**Purpose**: The built-in json library is used to save the extracted and structured data into a JSON file for further processing.

**Usage**: Serializes Python objects (such as the extracted data dictionary) into a JSON format.
Documentation: json


In [ ]:

#  Step 1: Data Extraction
# File paths
PDF_FILES = {
    "affiliated_colleges": "data1.pdf",
    "admission_notification": "data2.pdf"
}
CSV_FILE = "uok_data.csv.txt"


def extract_text_from_pdf(pdf_path):
    """Extract text from PDF files."""
    from pdfminer.high_level import extract_text
    try:
        return extract_text(pdf_path)
    except Exception as e:
        return str(e)


def extract_text_from_csv(csv_path):
    """Extract text from CSV file."""
    with open(csv_path, 'r', encoding='utf-8') as file:
        return file.read()


def generate_json():
    """Parse data from files and save as JSON."""
    data = {
        "affiliated_colleges": extract_text_from_pdf(PDF_FILES["affiliated_colleges"]),
        "admission_notification": extract_text_from_pdf(PDF_FILES["admission_notification"]),
        "general_info": extract_text_from_csv(CSV_FILE)
    }
    # Save the data as JSON
    with open("uok_data.json", "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print("uok_data.json has been created successfully!")

# Step 2: Load and Preprocess Data

This step involves loading the extracted data from the JSON file, splitting it into smaller chunks, and cleaning the data for further analysis.

**Libraries and Technologies Used:**
json (for loading the extracted data):

**Purpose:** The json library is used again to load the JSON file that was created in Step 1, which contains all the extracted data from PDFs and CSVs.

**Usage: **Loads the raw data into Python dictionaries or lists.

**Documentation:** json
pandas (for creating DataFrame and managing data):

**Purpose:** pandas is used to handle the data in a tabular format (DataFrame) for easier manipulation and processing.

**Usage:** Creates a structured DataFrame where each row contains a piece of text from the data and its corresponding label.

**Documentation:** pandas
spaCy (for text preprocessing and natural language processing):

**Purpose:** spaCy is a powerful NLP library that is used for cleaning the text data. This includes tasks like tokenization, lemmatization, and removing stopwords.

**Usage:** Used to preprocess the text by tokenizing and lemmatizing words, and removing unnecessary stopwords.

**Documentation:** spaCy
re (for text cleaning with regular expressions):

**Purpose:** The built-in re library is used to perform text cleaning tasks, like removing special characters, punctuation, and unwanted patterns.

**Usage:** Used for cleaning the text and removing any non-alphanumeric characters.
Documentation: re


In [ ]:
# ------------------ STEP 2: Load and Preprocess Data ------------------
def load_data():
    """Load data from uok_data.json and prepare labeled dataset."""
    with open("uok_data.json", "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    data = []
    labels = []

    # Affiliated colleges
    for line in raw_data["affiliated_colleges"].split("\n"):
        if line.strip():
            data.append(line.strip())
            labels.append("college_info")

    # Admission notifications
    for line in raw_data["admission_notification"].split("\n"):
        if line.strip():
            data.append(line.strip())
            labels.append("admission")

    # General info
    for line in raw_data["general_info"].split("."):
        if line.strip():
            data.append(line.strip())
            labels.append("general_info")

    return pd.DataFrame({"text": data, "label": labels})


def preprocess_text(text):
    """Clean and preprocess text using spaCy."""
    doc = nlp(text.lower())
    # Remove stopwords, punctuation, and non-alphanumeric tokens
    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens)



# Step 3: Train the ML Model

In this step, we train a machine learning model (Logistic Regression) using the preprocessed text data to classify user queries.

**Libraries and Technologies Used:**
scikit-learn (for machine learning model training and evaluation):

**Purpose:** scikit-learn is a comprehensive library for machine learning in Python. It is used here for model training, evaluation, and text vectorization.

**Usage:**
TF-IDF Vectorization: Converts text data into numerical form (vectors) that can be used by machine learning models.
Logistic Regression: Trains a classifier to predict the category of a given query (e.g., college_info, admission, etc.).

**Documentation:** scikit-learn
TfidfVectorizer (for converting text data into numerical vectors):

**Purpose:** This is a part of scikit-learn that converts text into numerical vectors using the Term Frequency-Inverse Document Frequency (TF-IDF) method.

**Usage:** Transforms the cleaned text into a matrix of TF-IDF features for model training.

**Documentation:** TfidfVectorizer
LogisticRegression (for training a classification model):

**Purpose:** LogisticRegression is used for text classification tasks. It is a simple yet effective classification algorithm for binary and multi-class classification problems.

**Usage:** Trains a logistic regression model to classify text into predefined categories.

**Documentation:** LogisticRegression
train_test_split (for splitting the data into training and testing sets):

**Purpose:** train_test_split splits the dataset into training and testing subsets to evaluate the performance of the model.
Usage: Splits the data into 80% training data and 20% testing data.

**Documentation:** train_test_split
accuracy_score (for model evaluation):

**Purpose:** This function calculates the accuracy of the model on the test data by comparing predicted values with actual values.

**Usage:** Used to evaluate the performance of the trained model.
Documentation: accuracy_score

In [ ]:
# ------------------ STEP 3: Train the ML Model ------------------

def train_model(df):
    """Train a logistic regression model using TF-IDF."""
    df["text_clean"] = df["text"].apply(preprocess_text)

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df["text_clean"])
    y = df["label"]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train logistic regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    print(f"Model Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

    return model, vectorizer


# Step 4: Chatbot Logic
This step involves detecting the language of the user’s query, matching it with pre-defined responses, and using the trained model for any remaining queries.

Libraries and Technologies Used:
langdetect (for language detection):

**Purpose**: The langdetect library is used to detect the language of the user's query.

**Usage**: Ensures that the chatbot only responds to queries in English.

**Documentation**: langdetect
spaCy (for text processing):

**Purpose**: spaCy is again used here for tokenization, lemmatization, and stopword removal in the chatbot logic.

**Usage**: Cleans the user’s input before feeding it to the model.

**Documentation**: spaCy
Custom Logic (for handling specific queries):

**Purpose**: Custom Python code to handle frequently asked questions (e.g., "Tell me about the university", "How many courses are available?").

**Usage**: Direct responses are returned if certain keywords are detected in the query.

In [ ]:
# ------------------ STEP 4: Chatbot Logic ------------------

def detect_language(query):
    """Detect the language of the user's query."""
    try:
        return detect(query)
    except:
        return "unknown"


def get_custom_response(query):
    """Provide specific responses for common questions."""
    query = query.lower()
    if "tell me about" in query:
        return "The department is devoted to impart knowledge as well as social values for overall development."
    elif "how many courses" in query:
        return "There are multiple courses, including programs in Arts, Science, Commerce, and more."
    elif "how many subjects" in query:
        return "Subjects include Chemistry, Physics, Mathematics, Botany, and Zoology, among others."
    elif "how many departments" in query:
        return "The university has departments like Science, Arts, Commerce, Law, and more."
    return None


def get_response(query, model, vectorizer, df):
    """Get a response based on the user's query."""
    lang = detect_language(query)
    if lang != "en":
        return "Sorry, I can only respond to queries in English."

    # Check for specific responses
    custom_response = get_custom_response(query)
    if custom_response:
        return custom_response

    # Use the ML model for general queries
    query_clean = preprocess_text(query)
    query_vectorized = vectorizer.transform([query_clean])
    predicted_label = model.predict(query_vectorized)[0]

    # Fetch a relevant response
    response_data = df[df["label"] == predicted_label]
    return response_data.sample(1)["text"].values[0]


def chat_with_user(model, vectorizer, df):
    """Interactive chat session."""
    print("Welcome to the University of Kota Chatbot!")
    print("Ask me anything about affiliated colleges, admissions, or general information.")
    print("Type 'exit' to end the chat.\n")

    while True:
        user_query = input("You: ")
        if user_query.lower() == "exit":
            print("Chatbot: Goodbye! Have a great day!")
            break

        # Get chatbot response
        response = get_response(user_query, model, vectorizer, df)
        print(f"Chatbot: {response}\n")


# Step 5: Main Function

This is the execution point where the above functions are brought together to create a chatbot application.

**Libraries and Technologies Used:**
Built-in Python Modules: Used for file handling (json, os), exception handling, and program flow control.

input(): For interactive user input during the chat session.

print(): For displaying chatbot responses in the console.


In [ ]:
# ------------------ STEP 5: Main Function ------------------

if __name__ == "__main__":
    try:
        # Generate JSON file from raw data
        print("Extracting data...")
        generate_json()

        # Load and preprocess data
        print("Loading data...")
        df = load_data()

        # Train ML model
        print("Training model...")
        model, vectorizer = train_model(df)

        # Start chatbot
        chat_with_user(model, vectorizer, df)

    except FileNotFoundError as e:
        print(f"File not found: {e}")
        print("Please ensure the required files are in the same directory.")

Extracting data...
uok_data.json has been created successfully!
Loading data...
Training model...
Model Accuracy: 84.73%
Welcome to the University of Kota Chatbot!
Ask me anything about affiliated colleges, admissions, or general information.
Type 'exit' to end the chat.

Chatbot: Objectives of NSS for Student Volunteers

understand the community in which they work
understand themselves in relation to their community;
identify the needs and problems of the community and involve them in problem solving process;
develop among themselves a sense of social and civic responsibility;
utilize their knowledge in finding practical solution to individual and community problems    
develop competence required for group living and sharing of responsibilities;
gain skills in mobilizing community participation;
acquire leadership qualities and democratic attitude;
develop capacity to meet emergencies and natural disasters
practice national integration and social harmony

Chatbot: There are multiple 